In [ ]:
import pandas as pd
import plotly.express as px
import matplotlib as plt
import numpy as np
import plotly.graph_objects as go
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Clean and FE

## Cleaning drafts data

In [ ]:
cbb = pd.read_csv('CollegeBasketballPlayers2009-2021.csv')
drafts = pd.read_excel('DraftedPlayers2009-2021.xlsx')
nba = pd.read_csv('player_data_03_22.csv')

<ipython-input-69-09bcbadbdcc5>:1: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  cbb = pd.read_csv('CollegeBasketballPlayers2009-2021.csv')


In [ ]:
# clean
drafts.shape

(781, 7)

In [ ]:
%%capture
drafts.columns = ['player', 'team', 'affiliation', 'year', 'round number', 'round pick', 'overall pick']
drafts = drafts.iloc[1:,:]
drafts['year'] = drafts['year'].astype(int)
drafts['round number'] = drafts['round number'].astype(int)
drafts['round pick'] = drafts['round pick'].astype(int)
drafts['overall pick'] = drafts['overall pick'].astype(int)

## Cleaning college data

In [ ]:
cbb.shape

(61061, 66)

In [ ]:
cbb = cbb.iloc[:,:64]

In [ ]:
print(cbb.isnull().sum())

player_name                            0
team                                   0
conf                                   0
GP                                     0
Min_per                                0
Ortg                                   0
usg                                    0
eFG                                    0
TS_per                                 0
ORB_per                                0
DRB_per                                0
AST_per                                0
TO_per                                 0
FTM                                    0
FTA                                    0
FT_per                                 0
twoPM                                  0
twoPA                                  0
twoP_per                               0
TPM                                    0
TPA                                    0
TP_per                                 0
blk_per                                0
stl_per                                0
ftr             

In [ ]:
cbb = cbb.drop(columns=cbb.loc[:,'Rec Rank':'pick'], axis=1)
cbb = cbb.dropna(subset=list(cbb.columns[34:]))

In [ ]:
cbb.shape

(61015, 52)

## Cleaning nba data

In [ ]:
nba.shape

(9660, 42)

In [ ]:
nba = nba.iloc[:,1:]

## Adding total season stats to college and nba data

In [ ]:
nba['total_PTS'] = round(nba['PTS'] * nba['G'])
nba['total_AST'] = round(nba['AST'] * nba['G'])
nba['total_REB'] = round(nba['TRB'] * nba['G'])
nba['total_STL'] = round(nba['STL'] * nba['G'])
nba['total_BLK'] = round(nba['BLK'] * nba['G'])

cbb['total_PTS'] = round(cbb['pts'] * cbb['GP'])
cbb['total_AST'] = round(cbb['ast'] * cbb['GP'])
cbb['total_REB'] = round(cbb['treb'] * cbb['GP'])
cbb['total_STL'] = round(cbb['stl'] * cbb['GP'])
cbb['total_BLK'] = round(cbb['blk'] * cbb['GP'])
cbb['FG%'] = (cbb['twoPM'] + cbb['TPM']) / (cbb['twoPA'] + cbb['TPA'])

## Names cleaning and features selection

In [ ]:
drafts['affiliation'] = drafts['affiliation'].str.replace('State', 'St.')

In [ ]:
nba = nba[['Player', 'Year', 'Pos', 'G', 'PTS', 'TRB', 'AST', 'STL', 'BLK',
           'FG%', '3P%', '2P%', 'FT%',
           'total_PTS', 'total_AST', 'total_REB', 'total_STL', 'total_BLK']]

cbb = cbb[['player_name', 'team', 'year', 'GP', 'treb', 'ast', 'stl', 'blk', 'pts',
           'FG%', 'TP_per', 'twoP_per', 'FT_per',
           'total_PTS', 'total_AST', 'total_REB', 'total_STL', 'total_BLK']]

nba = nba.rename(columns={'Player': 'player_name'})
cbb = cbb.rename(columns={'pts': 'PTS',
                          'treb': 'TRB',
                          'ast': 'AST',
                          'stl': 'STL',
                          'blk': 'BLK',
                          'TP_per': '3P%',
                          'twoP_per': '2P%',
                          'FT_per': 'FT%'})

print(nba.shape)
print(cbb.shape)

(9660, 18)
(61015, 18)


In [ ]:
## This filters the college and draft data to players that appeared in both

# renaming columns for join
drafts = drafts.rename(columns={'player': 'player_name',
                                'affiliation': 'college_team'})

cbb = cbb.rename(columns={'team': 'college_team'})

# we want unique combinations
cbb_names = cbb[['player_name', 'college_team']].drop_duplicates()

# generates combinations of plater name and team that appear in both
selection = pd.merge(drafts, cbb_names, on=['player_name', 'college_team'], how='inner')[['player_name', 'college_team']]
selection_list = np.array(selection).tolist()

# filter on college and draft data
cbb['combination'] = cbb.apply(lambda row: [row['player_name'], row['college_team']], axis=1)
cbb = cbb[cbb['combination'].isin(selection_list)]

drafts['combination'] = drafts.apply(lambda row: [row['player_name'], row['college_team']], axis=1)
drafts = drafts[drafts['combination'].isin(selection_list)]

In [ ]:
# filtering nba data
nba = nba[(nba['Year'] >= 2009) & (nba['Year'] <= 2021)]
nba = nba[nba['player_name'].isin(list(selection['player_name']))]

# potential issue of repeated player names in nba data

In [ ]:
drafts[drafts['player_name'] == 'Justin Jackson']

,player_name,team,college_team,year,round number,round pick,overall pick,combination
223,Justin Jackson,Denver Nuggets,Maryland,2018,2,13,43,"[Justin Jackson, Maryland]"
255,Justin Jackson,Portland Trail Blazers,North Carolina,2017,1,15,15,"[Justin Jackson, North Carolina]"


In [ ]:
drafts = drafts.drop(255)

In [ ]:
drafts[drafts['player_name'] == 'Justin Jackson']

,player_name,team,college_team,year,round number,round pick,overall pick,combination
223,Justin Jackson,Denver Nuggets,Maryland,2018,2,13,43,"[Justin Jackson, Maryland]"


# SQL database

In [ ]:
drafts = drafts.drop(columns='combination', axis=1)
cbb = cbb.drop(columns='combination', axis=1)

In [ ]:
import sqlite3

conn = sqlite3.connect('nba_college.db')
c = conn.cursor()

drafts.to_sql('draft', conn, if_exists='replace', index=False)
cbb.to_sql('cbb', conn, if_exists='replace', index=False)
nba.to_sql('nba', conn, if_exists='replace', index=False)

1976

In [ ]:
name = 'Stephen Curry'

draft_query = f"""
SELECT *
FROM draft
WHERE player_name = '{name}'
"""

cbb_query = f"""
SELECT *
FROM cbb
WHERE player_name = '{name}'
"""

nba_query = f"""
SELECT *
FROM nba
WHERE player_name = '{name}'
"""

In [ ]:
c.execute(draft_query)
results1 = c.fetchall()

In [ ]:
results1[0]

('Stephen Curry', 'Golden State Warriors', 'Davidson', 2009, 1, 7, 7)

In [ ]:
nba

,player_name,Year,Pos,G,PTS,TRB,AST,STL,BLK,FG%,3P%,2P%,FT%,total_PTS,total_AST,total_REB,total_STL,total_BLK
17,Ben Bentil,2017,PF,3,0.0,0.7,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.0,0.0,2.0,0.0,0.0
18,DeAndre Liggins,2017,SG,62,2.5,1.7,0.9,0.8,0.2,0.387,0.370,0.396,0.625,155.0,56.0,105.0,50.0,12.0
22,Dwight Powell,2017,C,77,6.7,4.0,0.6,0.8,0.5,0.515,0.284,0.571,0.759,516.0,46.0,308.0,62.0,38.0
23,Harrison Barnes,2017,PF,79,19.2,5.0,1.5,0.8,0.2,0.468,0.351,0.492,0.861,1517.0,118.0,395.0,63.0,16.0
28,Nerlens Noel,2017,C,51,8.7,5.8,1.0,1.3,1.0,0.595,0.000,0.597,0.694,444.0,51.0,296.0,66.0,51.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9648,John Henson,2017,C,58,6.8,5.1,1.0,0.5,1.3,0.515,0.000,0.516,0.692,394.0,58.0,296.0,29.0,75.0
9649,Khris Middleton,2017,SF,29,14.7,4.2,3.4,1.4,0.2,0.450,0.433,0.459,0.880,426.0,99.0,122.0,41.0,6.0
9650,Malcolm Brogdon,2017,SG,75,10.2,2.8,4.2,1.1,0.2,0.457,0.404,0.480,0.865,765.0,315.0,210.0,82.0,15.0
9657,Terrence Jones,2017,PF,54,10.8,5.7,1.1,0.7,1.0,0.470,0.253,0.508,0.606,583.0,59.0,308.0,38.0,54.0


# Dashboard Testing

## Top information

In [ ]:
name = 'Stephen Curry'

draft_query = f"""
SELECT year, [round number], [round pick]
FROM draft
WHERE player_name = '{name}'
"""

c.execute(draft_query)
results1 = c.fetchall()

year = results1[0][0]
round_number = results1[0][1]
round_pick = results1[0][2]


seasons_cbb_query = f"""
SELECT count(*)
FROM cbb
WHERE player_name = '{name}'
"""

c.execute(seasons_cbb_query)
results2 = c.fetchall()

cbb_seasons = results2[0][0]


seasons_nba_query = f"""
SELECT count(*)
FROM nba
WHERE player_name = '{name}'
"""

c.execute(seasons_nba_query)
results3 = c.fetchall()

nba_seasons = results3[0][0]

## Line plot

In [ ]:
nba_stats_query = f"""
SELECT Year, PTS, AST, TRB, STL, BLK
FROM nba
WHERE player_name = '{name}'
"""

c.execute(nba_stats_query)
results5 = c.fetchall()
results5 = sorted(results5, key=lambda x: x[0])
years = [i[0] for i in results5]
PTS = [i[1] for i in results5]
AST = [i[2] for i in results5]
TRB = [i[3] for i in results5]
STL = [i[4] for i in results5]
BLK = [i[5] for i in results5]

cbb_averages_query = f"""
SELECT AVG(PTS), AVG(AST), AVG(TRB), AVG(STL), AVG(BLK)
FROM cbb
WHERE player_name = '{name}'
"""

c.execute(cbb_averages_query)
results6 = c.fetchall()
cPTS = results6[0][0]
cAST = results6[0][1]
cTRB = results6[0][2]
cSTL = results6[0][3]
cBLK = results6[0][4]

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=years, y=PTS, mode='lines+markers', name='nba PTS', line=dict(width=3), marker=dict(size=10)))
fig.add_trace(go.Scatter(x=years, y=[cPTS]*len(PTS), mode='lines', name='cbb PTS avg', line=dict(width=3)))
fig.update_layout(title='Points Comparison', xaxis_title='Years', yaxis_title='PTS')
fig.show()

In [ ]:
names_query = """
SELECT DISTINCT player_name
FROM draft
"""

c.execute(names_query)
results6 = c.fetchall()
names = [i[0] for i in results6]

In [ ]:
names

['Cade Cunningham',
 'Scottie Barnes',
 'Jalen Suggs',
 'Franz Wagner',
 'Davion Mitchell',
 'Ziaire Williams',
 'James Bouknight',
 'Joshua Primo',
 'Chris Duarte',
 'Moses Moody',
 'Corey Kispert',
 'Trey Murphy III',
 'Tre Mann',
 'Kai Jones',
 'Jalen Johnson',
 'Keon Johnson',
 'Isaiah Jackson',
 'Josh Christopher',
 'Quentin Grimes',
 'Jaden Springer',
 "Day'Ron Sharpe",
 'Jeremiah Robinson-Earl',
 'Jason Preston',
 'Herbert Jones',
 'Miles McBride',
 'JT Thor',
 'Ayo Dosunmu',
 'Neemias Queta',
 'Jared Butler',
 'Joe Wieskamp',
 'Isaiah Livers',
 'Greg Brown III',
 'Kessler Edwards',
 'Dalano Banton',
 'David Johnson',
 'Sharife Cooper',
 'Marcus Zegarowski',
 'Brandon Boston Jr.',
 'Luka Garza',
 'Charles Bassey',
 'Sandro Mamukelashvili',
 'Aaron Wiggins',
 'Scottie Lewis',
 'Balsa Koprivica',
 'Jericho Sims',
 'Anthony Edwards',
 'James Wiseman',
 'Patrick Williams',
 'Isaac Okoro',
 'Obi Toppin',
 'Jalen Smith',
 'Devin Vassell',
 'Tyrese Haliburton',
 'Kira Lewis Jr.',
 'Aar